In [28]:
import pandas as pd
import seaborn as sns

In [33]:
db = pd.read_csv('db_files/db_cleaned.csv')

In [34]:
db.shape

(558172, 21)

In [35]:
db.head(20)

,Unnamed: 0,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,...,ordering,pnconst,pcategory,region,types,nconst,primaryName,birthYear,deathYear,primaryProfession
0,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,10,nm0107463,editor,IT,imdbDisplay,nm0179321,Noreen Corcoran,1943,2016,"actress,soundtrack"
1,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,10,nm0107463,editor,IT,imdbDisplay,nm0228072,Françoise Diot,1914,2013,"editor,editorial_department,sound_department"
2,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,10,nm0107463,editor,IT,imdbDisplay,nm0755977,Maksim Sagalovich,1915,1997,writer
3,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,10,nm0107463,editor,IT,imdbDisplay,nm0790590,Haleigh Sheehan,1983,2024,actress
4,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,10,nm0107463,editor,IT,imdbDisplay,nm0866390,Peter Tomarken,1942,2006,"actor,producer,writer"
5,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,2,nm0413168,actor,EG,imdbDisplay,nm0179321,Noreen Corcoran,1943,2016,"actress,soundtrack"
6,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,2,nm0413168,actor,EG,imdbDisplay,nm0228072,Françoise Diot,1914,2013,"editor,editorial_department,sound_department"
7,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,2,nm0413168,actor,EG,imdbDisplay,nm0755977,Maksim Sagalovich,1915,1997,writer
8,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,2,nm0413168,actor,EG,imdbDisplay,nm0790590,Haleigh Sheehan,1983,2024,actress
9,34803,tt0035423,Kate & Leopold,Kate & Leopold,0,2001,2001,118,"Comedy,Fantasy,Romance",6.4,...,2,nm0413168,actor,EG,imdbDisplay,nm0866390,Peter Tomarken,1942,2006,"actor,producer,writer"
